In [1]:
import Turtle_trading
import pandas as pd 
import numpy as np


In [2]:
class Account_Handler:
    
    def __init__(self,account_size,risk_percentage):
        self.account_size = account_size
        self.risk_percentage = risk_percentage/100
        self.risk = self.account_size * self.risk_percentage

    def cal_positon_size(self,filename,ticker):
        ticker = ticker.upper()
        xls = pd.ExcelFile(filename)
        if ticker in xls.sheet_names:
            df = pd.read_excel(filename, sheet_name=ticker)

            # Ensure Date column is datetime and set as index
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)

            last_row = df.iloc[-1]
            try:
                ATR = last_row['Avg_True_Range']
            except KeyError as msg:
                return msg

            position_size = self.risk // ATR
            return int(position_size)
        else:
            return f"Ticker {ticker} not found in the Excel file."


In [6]:
filename = r"C:\Users\DELL\open_source\Turtle_trading\default2.xlsx"

A1 = Account_Handler(account_size=100000, risk_percentage=1)
A1.cal_positon_size(filename= filename, ticker="suzlon")

435